<a href="https://colab.research.google.com/github/goelnikhils-lgtm/languagemodels/blob/main/OpenAI_Multi_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#agent system built using open AI SDK
#code credit -> https://www.youtube.com/watch?v=LuA4DLbZIpI

In [ ]:
!pip install agents
import os
import getpass
import openai
from dotenv import load_dotenv


load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
#Setup the Google Calendar Authorization
print("\n Follow the instructions above to get your Google Calendar access token.")
os.environ["GOOGLE_CALENDAR_ACCESS_TOKEN"] = getpass.getpass("Enter your Google Calendar access token: ")


In [ ]:
import asyncio
import json
from datetime import datetime, timedelta
from typing import Any, Dict, List, Optional, Union
from pydantic import BaseModel
from agents import Agent ,HostedMCPTool,WebSearchTool,Runner , ModelSettings
from openai.types.shared.reasoning import Reasoning


In [7]:
#
class CalendarEvent(BaseModel):
    """ Represents a calendar event with essential details"""
    title: str
    start_time: str
    end_time: str
    location: Optional[str] = None
    attendees: Optional[List[str]] = None
    description: Optional[str] = None

class ResearchTopic(BaseModel):
  """A specific topic to research related to an event."""
  topic: str
  relevance:str
  search_query:str

class EventResearch(BaseModel):
  """ Research Findings for a calendar event"""
  event_title:str
  event_type:str
  key_topics:List[str]
  research_queries:List[ResearchTopic]

class PreparationItem(BaseModel):
  """ A specific preparation action item."""
  action:str
  priority:str
  time_estimate:List[str]
  resources:List[str]

class PreparationGuide(BaseModel):
  """ Complete preparation guide for an event."""
  event_title:str
  event_time:str
  summary:str
  key_insights:List[str]
  preparation_checklist:List[PreparationItem]
  talking_points:List[str]
  questions_to_ask:List[str]
  potential_challenges:List[str]
  recommended_readings:List[str]

In [ ]:
#Create the Calendar Agent
calendar_agent = Agent(
    name = "CalendarAssistant",
    instructions = """You are a calendar assistant that helps users fetch and understand their schedule.
    When asked about events for a specific day , retrieve all calendar events for that day and provide a clear ,
    structured summary of the schedule. Include event titles , times , locations, and attendees.""",
    tools = [
        HostedMCPTool(
            tool_config = {
                "type" :  "mcp",
                "server_label" : "google_calendar",
                "connector_id" : "connector_googlecalendar",
                "api_key" : os.getenv("GOOGLE_CALENDAR_ACCESS_TOKEN"),
                "require_approval":"never"
            }
        )
    ],
    model ="gpt-5",
)


In [ ]:
#Create the Event Analyzer Agent
event_analyzer_agent = Agent(
    name = "EventAnalyzer",
    instructions = """You are an expert at analyzing calendar events to determine what preparation would be required.
    Given a calendar event , analyze its title , description , attendees, and context to:
    1.Identify the type of event (meeting , presentation , interview , workshop, etc.)
    2.Extract key topics that should be researched
    3.Generate 3-5 specific ,actionable research queries that would help someone prepare

    Focus on practical , relevant research that would actually help in the meeting:
    For example:
    - For a product meeting: latest features, competitor analysis , user feedback
    - For a technical discussion: relevant technologies , best practices , recent developments
    - For a client meeting: company background , recent news  , industry trends

    Make search queries specific and likely to return useful results.""",
    model ="gpt-5",
    model_settings = ModelSettings(reasoning = Reasoning(effort = "medium")),
    output_type = EventResearch # to get structured output from Model. Type Pydantic Class
)

In [ ]:
#Create the Research  Agent
research_agent = Agent(
    name = "ResearchAgent",
    instructions = """You are research assistant specializing in gathering relevant information for meeting.
    Given a search query , perform a comprehensive web search and provide a concise summary of the findings:

    Focus on:
    - Recent Information (priortize content for last 6 months)
    - Authoritative sources
    - Practical Insights that would be useful  in a meeting context
    - Key , Facts , Trends , and developments

    Your summary should be 2-3 paragraphs , highlighting the most important and actionable information.
    Always cite sources when possible and note any particularly interesting or surprising findings""",
    model ="gpt-4.1",
    tools = [WebSearchTool()],
    model_settings = ModelSettings(tool_choice = "required")
    )

In [ ]:
#Create the Research  Agent
preparation_guide_agent = Agent(
    name = "PreparationGuideAgent",
    instructions = """You are an expert meeting preparation assistant.
    Given event details and research findins create a detailed preparation guide that helps someone:

    Your guide should be :
    - Practical and actionable with specific steps
    - Prioritized by importance and urgency
    - Time-conscious (realistic time estimates)
    - Comprehensive but not overwhelming

    Include:
    - A concise summary of what the meeting is about
    - Key insights from the research
    - Specific preparation actions with time estimates
    - Smart talking points based on the research
    - Thoughtful questions to ask during the meeting
    - Potential challenges or sensitive topics to be aware of
    - Links or resources  for deeper reading
    Tailor the guide to the specific type of event(presentation , interview , client meeting,etc.).""",
    model ="gpt-5",
    model_settings = ModelSettings(reasoning = Reasoning(effort = "high"),
    output_type = PreparationGuide, # to get structured output from Model. Type Pydantic Class
    )


In [14]:
#orchestrator class
class CalendarOrchestrator:
  """ Orchestrates the calendar research workflow. """
  def __init__(self):
    self.calendar_agent = calendar_agent
    self.analyzer_agent = event_analyzer_agent
    self.research_agent = research_agent
    self.preparation_guide_agent = preparation_guide_agent

  async def research_event(self,event_info:str) ->dict:
      """  Research a single calendar event """
      print(f"\n Analyzing event...")

      #Note: Context Engineering - The event_info string provides raw context that gets
      #progressively refined through each agent interaction

      #Analyze the event to determine research topics
      analysis_result = await Runner.run(
          self.analyzer_agent,
          f"Analyze this calendar event and determine what research would be helpful:\n\n{event_info}"
        )
      event_research = analysis_result.final_output_as(EventResearch)

      #Note Context Engineering: Parallel research maintains independent context
      #for each query while preserving the shared event context, allowing focused
      #yet coordinated information gathering

      #Perform research on all topics in parallel
      print(f"\n Researching {len(event_research.research_queries)} topics...")
      research_tasks = []
      for topic in event_research.research_queries:
        task = asyncio.create_task(
            self.perform_research(topic.search_query))
        research_tasks.append(task)

      research_results = await asyncio.gather(*research_tasks)
      #Combine all research into a particular guide
      print(f"\n Creating Preparation Guide...")
      research_summary = "\n\n".join([result for result in research_results if result])

      #Note Context Engineering: This structured prompt template assembles multiple
      #context sources (event details , analysis ,research) into a rich , multi-layered
      #context that enables comprehensive synthesis by the guide agent
      guide_input = f"""
      Event Details:
      {event_info}

      Event Analysis:
      Type of Event: {event_research.event_type}
      Key Topics: {",".join(event_research.key_topics)}

      Research Findings:
      {research_summary}
      """
      #Create the preparation guide
      guide_result = await Runner.run(
          self.preparation_guide_agent,
          guide_input
      )
      return{
          "event":event_info,
          "analysis":event_research,
          "research":research_results,
          "guide":guide_result.final_output_as(PreparationGuide)
      }
  async def perform_research(self,query:str) ->str:
    """ Perform a single research query."""
    try:
      result = await Runner.run(
          self.research_agent,
          f"Perform a comprehensive web search and provide a concise summary of the findings:\n\n{query}"
      )
      return str(result.final_output)
    except Exception as e:
      print(f"Research Error: {e}")
      return None

  async def prepare_for_date(self,date:str) ->List[dict]:
    """ Perform for all events on a specific date."""
    #fetch calendar events
    calendar_result = await Runner.run(
          self.calendar_agent,
          f"Get all my calendar events for {date}. Include event titles , times , locations , attendees :\n\n")
    events_text = str(calendar_result.final_output)
    #Parse events
    if "no_events" in events_text.lower() or "empty" in events_text.lower():
        return []

    results = []
    result = await self.research_event(events_text)
    results.append(result)
    return results

In [ ]:
#run the agents now
orchestrator = CalendarOrchestrator()

#get today's date
today = datetime.now().strftime("%Y-%m-%d")
print(f"Preparing for events on:{today}")

#run the preparation flow
async def run_preparation():
  results = await orchestrator.prepare_for_date(today)
  if not results:
    print("No events found for today.")
    return

  #Display results for each event
  for i , result in enumerate(results,1):
    print(f"\n {"="*60}")
    print(f"EVENT {i} PREPARATION GUIDE")
    print("="*60")
    guide = result["guide"]

    print(f"Event Title: {guide.event_title}")
    print(f"Event Time: {guide.event_time}")
    print(f"Summary: {guide.summary}")

    print(f"Key Insights")
    for insight in guide.key_insights: #Show top 3 insights
      print(f"- {insight}")

    print(f"Top Preparation Actions")
    for item in guide.preparation_checklist[:3]:
      print(f"[{item.priority.upper()}] {item.action} ({item.time_estimate})")

    print(f" Suggested Talking Points")
    for point in guide.talking_points[:3]:
      print(f"- {point}")

    print(f"Questions to Ask")
    for question in guide.questions_to_ask[:3]:
      print(f"- {question}")

#run the async function
await run_preparation()
